In [63]:
import pandas as pd
pd.options.display.max_columns=1000
pd.options.display.width=200
pd.options.display.min_rows=60
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn import preprocessing

from datetime import datetime

In [64]:
filename = 'clinvar_conflicting.csv'
df =  pd.read_csv(filename)
df.head()

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,CLNDNINCL,CLNHGVS,CLNSIGINCL,CLNVC,CLNVI,MC,ORIGIN,SSR,CLASS,Allele,Consequence,IMPACT,SYMBOL,Feature_type,Feature,BIOTYPE,EXON,INTRON,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,DISTANCE,STRAND,BAM_EDIT,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955563G>C,NaN,single_nucleotide_variant,NaN,SO:0001583|missense_variant,1.0,NaN,0,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,protein_coding,1/39,NaN,61,11,4,R/P,cGg/cCg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421,11.390,1.133255,-2.0
1,1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.955597G>T,NaN,single_nucleotide_variant,NaN,SO:0001819|synonymous_variant,1.0,NaN,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,protein_coding,1/39,NaN,95,45,15,P,ccG/ccT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421,8.150,0.599088,NaN
2,1,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.955619G>C,NaN,single_nucleotide_variant,NaN,SO:0001583|missense_variant,1.0,NaN,1,C,missense_variant,MODERATE,AGRN,Transcript,NM_001305275.1,protein_coding,1/39,NaN,117,67,23,V/L,Gtc/Ctc,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421,3.288,0.069819,1.0
3,1,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",NaN,NC_000001.10:g.957640C>T,NaN,single_nucleotide_variant,NaN,SO:0001819|synonymous_variant,1.0,NaN,0,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,protein_coding,2/39,NaN,311,261,87,D,gaC/gaT,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421,12.560,1.356499,NaN
4,1,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,NaN,not_specified,NaN,NC_000001.10:g.976059C>T,NaN,single_nucleotide_variant,NaN,SO:0001819|synonymous_variant,1.0,NaN,1,T,synonymous_variant,LOW,AGRN,Transcript,NM_001305275.1,protein_coding,4/39,NaN,576,526,176,L,Ctg/Ttg,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.421,17.740,2.234711,NaN


In [65]:
def compare_dicts(a,b,ignore=['test_score','train_score','tn','fn','tp','fp','f1_score','precision','recall']):
    a = dict(a)
    b = dict(b)
    for k in ignore:
        a.pop(k,None)
        b.pop(k,None)
        
    return tuple(a.items()) == tuple(b.items())

In [66]:
hyperparam_table = []

In [67]:
filename = 'clinvar_conflicting.csv'
df =  pd.read_csv(filename)

#select only columns with int or float data types
df = df.select_dtypes(['number'])
#drop any columns with null values
df.dropna(axis=1,inplace=True)
#remove the class series
var_class = df.pop('CLASS')

x_train,x_test,y_train,y_test = train_test_split(df,var_class,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)


exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression()
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df.shape),
                          'scaling':'no_scaling',
                            'filename': filename,
                            'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall


/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
df_hyp = pd.DataFrame(hyperparam_table)

In [69]:
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.0,0.0,0.0


In [70]:
filename = 'data_cleaned1.csv'
df1 =  pd.read_csv(filename)

#select only columns with int or float data types
df1 = df1.select_dtypes(['number'])
#drop any columns with null values
df1.dropna(axis=1,inplace=True)
#remove the class series
var_class1 = df1.pop('CLASS')

x_train,x_test,y_train,y_test = train_test_split(df1,var_class1,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)


exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df1.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression()
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df1.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                        'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [71]:
filename = 'data_cleaned1.csv'
df1 =  pd.read_csv(filename)

#select only columns with int or float data types
df1 = df1.select_dtypes(['number'])
#drop any columns with null values
df1.dropna(axis=1,inplace=True)
#remove the class series
var_class1 = df1.pop('CLASS')

x_train,x_test,y_train,y_test = train_test_split(df1,var_class1,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)


exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df1.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                 'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df1.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                          'class_weight':'balanced',
                        'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [72]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.00000,0.000000,0.0,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.00000,0.000000,0.0,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.40898,0.257055,1.0,balanced


In [73]:
filename = 'clinvar_conflicting.csv'
df =  pd.read_csv(filename)

#select only columns with int or float data types
df = df.select_dtypes(['number'])
#drop any columns with null values
df.dropna(axis=1,inplace=True)
#remove the class series
var_class = df.pop('CLASS')

scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df,columns=df.columns)

x_train,x_test,y_train,y_test = train_test_split(scaled_df,var_class,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_scaledxtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_scaledxtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_scaledytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_scaledytest.csv',index=False,header=False)


exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df.shape),
                                'scaling':'min_max',
                                'filename': filename,
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression()
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df.shape),
                          'scaling':'min_max',
                         'filename': filename,
                         'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.00000,0.000000,0.0,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.00000,0.000000,0.0,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.40898,0.257055,1.0,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.00000,0.000000,0.0,NaN


In [75]:
filename = 'clinvar_conflicting.csv'
df =  pd.read_csv(filename)

#select only columns with int or float data types
df = df.select_dtypes(['number'])
#drop any columns with null values
df.dropna(axis=1,inplace=True)
#remove the class series
var_class = df.pop('CLASS')

scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df,columns=df.columns)

x_train,x_test,y_train,y_test = train_test_split(scaled_df,var_class,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_scaledxtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_scaledxtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_scaledytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_scaledytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df.shape),
                                'scaling':'min_max',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df.shape),
                          'scaling':'min_max',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [76]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced


In [77]:
filename = 'data_cleaned1.csv'
df1 =  pd.read_csv(filename)

#select only columns with int or float data types
df1 = df1.select_dtypes(['number'])
#drop any columns with null values
df1.dropna(axis=1,inplace=True)
#remove the class series
var_class1 = df1.pop('CLASS')

scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df1)
scaled_df = pd.DataFrame(scaled_df,columns=df1.columns)

x_train,x_test,y_train,y_test = train_test_split(scaled_df,var_class1,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_scaledxtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_scaledxtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_scaledytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_scaledytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df1.shape),
                                'scaling':'min_max',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'}) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df1.shape),
                          'scaling':'min_max',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'}]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [78]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced


In [79]:
filename = 'data_cleaned3.csv'
df2 =  pd.read_csv(filename)

#select only columns with int or float data types
df2 = df2.select_dtypes(['number'])
#drop any columns with null values
df2.dropna(axis=1,inplace=True)
#remove the class series
var_class2 = df2.pop('CLASS')


x_train,x_test,y_train,y_test = train_test_split(df2,var_class2,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df2.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression()
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df2.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [80]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN


In [81]:
filename = 'data_cleaned3.csv'
df2 =  pd.read_csv(filename)

#select only columns with int or float data types
df2 = df2.select_dtypes(['number'])
#drop any columns with null values
df2.dropna(axis=1,inplace=True)
#remove the class series
var_class2 = df2.pop('CLASS')


x_train,x_test,y_train,y_test = train_test_split(df2,var_class2,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df2.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df2.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [82]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
7,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.544479,0.532021,1344,1078,407,431,0.367277,0.285620,0.514320,balanced


In [83]:
filename = 'data_cleaned3.csv'
df2 =  pd.read_csv(filename)

#select only columns with int or float data types
df2 = df2.select_dtypes(['number'])
#drop any columns with null values
df2.dropna(axis=1,inplace=True)
#remove the class series
var_class2 = df2.pop('CLASS')

scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df2)
scaled_df = pd.DataFrame(scaled_df,columns=df2.columns)

x_train,x_test,y_train,y_test = train_test_split(scaled_df,var_class2,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_scaledxtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_scaledxtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_scaledytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_scaledytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df2.shape),
                                'scaling':'min_max',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df2.shape),
                          'scaling':'min_max',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [84]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
7,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.544479,0.532021,1344,1078,407,431,0.367277,0.285620,0.514320,balanced
8,0.05,0,"(65188, 47)",min_max,data_cleaned3.csv,sklearn.LogisticRegression,0.520245,0.525530,1103,1319,245,593,0.431273,0.310146,0.707637,balanced


In [85]:
filename = 'data_cleanednlp2.csv'
df3 =  pd.read_csv(filename,index_col=0)

#select only columns with int or float data types
df3 = df3.select_dtypes(['number'])
#drop any columns with null values
df3.dropna(axis=1,inplace=True)
#remove the class series
var_class3 = df3.pop('CLASS')

scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df3)
scaled_df = pd.DataFrame(scaled_df,columns=df3.columns)

x_train,x_test,y_train,y_test = train_test_split(scaled_df,var_class3,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_scaledxtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_scaledxtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_scaledytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_scaledytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df3.shape),
                                'scaling':'min_max',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df3.shape),
                          'scaling':'min_max',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [86]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(15)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
7,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.544479,0.532021,1344,1078,407,431,0.367277,0.285620,0.514320,balanced
8,0.05,0,"(65188, 47)",min_max,data_cleaned3.csv,sklearn.LogisticRegression,0.520245,0.525530,1103,1319,245,593,0.431273,0.310146,0.707637,balanced
9,0.05,0,"(65188, 55)",min_max,data_cleanednlp2.csv,sklearn.LogisticRegression,0.515031,0.522785,1092,1330,251,587,0.426134,0.306208,0.700477,balanced


In [87]:
filename = 'data_cleanednlp2.csv'
df3 =  pd.read_csv(filename,index_col=0)

#select only columns with int or float data types
df3 = df3.select_dtypes(['number'])
#drop any columns with null values
df3.dropna(axis=1,inplace=True)
#remove the class series
var_class3 = df3.pop('CLASS')



x_train,x_test,y_train,y_test = train_test_split(df3,var_class3,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df3.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'class_weight':'balanced',
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression(class_weight='balanced')
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df3.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                          'class_weight':'balanced',
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [88]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(15)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
7,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.544479,0.532021,1344,1078,407,431,0.367277,0.285620,0.514320,balanced
8,0.05,0,"(65188, 47)",min_max,data_cleaned3.csv,sklearn.LogisticRegression,0.520245,0.525530,1103,1319,245,593,0.431273,0.310146,0.707637,balanced
9,0.05,0,"(65188, 55)",min_max,data_cleanednlp2.csv,sklearn.LogisticRegression,0.515031,0.522785,1092,1330,251,587,0.426134,0.306208,0.700477,balanced


In [89]:
filename = 'data_cleanednlp2.csv'
df3 =  pd.read_csv(filename,index_col=0)

#select only columns with int or float data types
df3 = df3.select_dtypes(['number'])
#drop any columns with null values
df3.dropna(axis=1,inplace=True)
#remove the class series
var_class3 = df3.pop('CLASS')

x_train,x_test,y_train,y_test = train_test_split(df3,var_class3,test_size=0.05,random_state=0)
x_train.to_csv(filename[:-4]+'_xtrain.csv',index=False)
x_test.to_csv(filename[:-4]+'_xtest.csv',index=False)
y_train.to_csv(filename[:-4]+'_ytrain.csv',index=False,header=False)
y_test.to_csv(filename[:-4]+'_ytest.csv',index=False,header=False)

exists = any([compare_dicts(a,b={'test_size': 0.05, 
                                'random_state': 0, 
                                'data_size': str(df3.shape),
                                'scaling':'no_scaling',
                                'filename': filename,
                                'model': 'sklearn.LogisticRegression'
                                }) 
              for a in hyperparam_table])

logisticRegr = LogisticRegression()
if not exists:
    hyperparam_table += [{'test_size': 0.05, 
                          'random_state': 0, 
                          'data_size': str(df3.shape),
                          'scaling':'no_scaling',
                          'filename': filename,
                          'model': 'sklearn.LogisticRegression'
                         }]
        
    logisticRegr.fit(x_train,y_train)
    
    predictions_test = logisticRegr.predict(x_test)
    predictions_train = logisticRegr.predict(x_train)
    
    score = logisticRegr.score(x_test,y_test)
    hyperparam_table[-1]['test_score'] = score
    
    training_score = logisticRegr.score(x_train,y_train)
    hyperparam_table[-1]['train_score'] = training_score
    
    tn, fp, fn, tp = confusion_matrix(y_test,predictions_test).ravel()
    hyperparam_table[-1]['tn'] = tn
    hyperparam_table[-1]['fp'] = fp
    hyperparam_table[-1]['fn'] = fn
    hyperparam_table[-1]['tp'] = tp
    
    f1 = f1_score(y_test,predictions_test)
    hyperparam_table[-1]['f1_score'] = f1
    precision = precision_score(y_test,predictions_test)
    hyperparam_table[-1]['precision'] = precision
    recall = recall_score(y_test,predictions_test)
    hyperparam_table[-1]['recall'] = recall

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [90]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(15)

,test_size,random_state,data_size,scaling,filename,model,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,class_weight
0,0.05,0,"(65188, 4)",no_scaling,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
1,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
2,0.05,0,"(65188, 44)",no_scaling,data_cleaned1.csv,sklearn.LogisticRegression,0.257055,0.251841,0,2422,0,838,0.408980,0.257055,1.000000,balanced
3,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
4,0.05,0,"(65188, 4)",min_max,clinvar_conflicting.csv,sklearn.LogisticRegression,0.395399,0.393457,512,1910,61,777,0.440851,0.289170,0.927208,balanced
5,0.05,0,"(65188, 44)",min_max,data_cleaned1.csv,sklearn.LogisticRegression,0.498466,0.505781,1025,1397,238,600,0.423280,0.300451,0.715990,balanced
6,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.742945,0.748159,2422,0,838,0,0.000000,0.000000,0.000000,NaN
7,0.05,0,"(65188, 47)",no_scaling,data_cleaned3.csv,sklearn.LogisticRegression,0.544479,0.532021,1344,1078,407,431,0.367277,0.285620,0.514320,balanced
8,0.05,0,"(65188, 47)",min_max,data_cleaned3.csv,sklearn.LogisticRegression,0.520245,0.525530,1103,1319,245,593,0.431273,0.310146,0.707637,balanced
9,0.05,0,"(65188, 55)",min_max,data_cleanednlp2.csv,sklearn.LogisticRegression,0.515031,0.522785,1092,1330,251,587,0.426134,0.306208,0.700477,balanced


In [91]:
time = datetime.now().strftime("%Y%m%d_%H:%M:%S")
filename = 'hyperparameter_table'+time+'.csv'
df_hyp.to_csv(filename,index=False)